<a href="https://colab.research.google.com/github/williamcarp/Cox_Ross_Rubenstein_Model/blob/main/CRR_trinomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###################################################################
# COX-ROSS-RUBENSTEIN (CRR) Trinomial Tree Implementation 
###################################################################

# Author      : Will Carpenter 
# Date Created: April 1st, 2021  

# Description: Trinomial tree implementation that prices European and 
# American calls/puts. With increasing tree step size (dt), the tree
# prices converge to Black-Scholes formulaic prices. Option data for 
# pricing Euro calls/puts can be obtained from Yahoo Finance. See the 
# PDF document labeled "CRR_Trinomial_Tree_Documentation" for a detailed
# discussion of the model and underlying asset-pricing theory. 

import math 
import numpy as np 
from scipy.stats import norm # cumulative normal distribution

In [ ]:
def crr_trinomial_tree(S, K, r, T, t, v, x):

    # Add new arguments for "call", "put", "euro", "amer"
    # Make everything American? make excercise = -Inf if European

    # S: initial asset price
    # K: strike price 
    # r: riskless rate 
    # T: time to maturity (in yrs.)
    # t: number of steps 
    # v: annualized volatility
    # x: euro call (= 1) euro put (= -1)

    # Calculate time increment 
    dt = T / t
    # Initialize tree  
    crrTree        = np.empty((2*t-1,t))
    crrTree[:]     = np.nan

    # Initialize tree parameters 
    u = math.exp(v*math.sqrt(2*dt))
    d = 1/u
    m = 1
    # Probability-up
    pu = ((math.exp(r*dt/2) - math.exp(-1*v*math.sqrt(dt/2))) / 
         (math.exp(v*math.sqrt(dt/2)) - math.exp(-1*v*math.sqrt(dt/2))))**2
    # Probability-down
    pd = ((math.exp(v*math.sqrt(dt/2)) - math.exp(r*dt/2)) /
         (math.exp(v*math.sqrt(dt/2)) - math.exp(-1*v*math.sqrt(dt/2))))**2
    # Probability-middle
    pm = 1 - (pu + pd)

    lastCol = crrTree.shape[1]-1
    mid     = len(crrTree)//2

    # Set price at middle of tree 
    crrTree[mid, lastCol] = max(x*S-x*K, 0)

    for i in range(1, mid+1):
        crrTree[mid-i, lastCol] = max(x*S*u**i - x*K, 0)
        crrTree[mid+i, lastCol] = max(x*S*d**i - x*K, 0)

    for col in range(lastCol-1, -1, -1):
        for row in range(0, col*2+1):
            # move backwards from previous prices 
            Su = crrTree[row,   col+1]
            Sm = crrTree[row+1, col+1]
            Sd = crrTree[row+2, col+1]
            # Calcuate price on tree
            crrTree[row, col] = math.exp(-r*dt)*(pu*Su + pm*Sm + pd*Sd)

    # print("u parameter: " "{:6.4f}".format(u))
    # print("d parameter: " "{:6.4f}".format(d))
    # print("pu parameter: " "{:5.3f}".format(pu))
    # print("pd parameter: " "{:5.3f}".format(pd))
    # print("pm parameter: " "{:5.3f}".format(pm))
    # print("pm+pu+pd: " "{:9.3f}".format(pm+pu+pd))

    # print("\nCRR Stock Price Tree:\n")
    # for i in crrTree:
    #     for j in i:
    #         print("{:7.2f}".format(j), end=" ")
    #     print() 
    # print("\n")

    return crrTree[0,0]



In [ ]:
# Black-Scholes Model

def black_scholes(S, K, r, T, t, v, x):

    d1 = (math.log(S/K) + (v**2/2 + r)*T)/(v*math.sqrt(T))
    d2 = d1 - v*math.sqrt(T)

    return x*S*norm.cdf(d1) - x*K*math.exp(-r*T)*norm.cdf(d2)
    

In [ ]:
# Testing for parameters and tree structure (see commented code above)

crr_price = crr_trinomial_tree(91.36, 95.00, 0.007, 15/252, n, 0.8218, 1)


In [ ]:
# Calculating Option Prices for various securities and compare results to 
# Black-Scholes (B-S) analytical pricing. 

# Relevant option data (current price, strike, term, and volatility) can be 
# obtained from Yahoo finance for a variety of different calls and puts. 

n = 300 # number of steps

# Monster Beverage 
crr_price = crr_trinomial_tree(91.36, 95.00, 0.007, 15/252, n, 0.8218, 1)
bs_price  =     black_scholes(91.36, 95.00, 0.007, 15/252, n, 0.8218, 1)
print("________________________________________________________________\n")
print("Monster Beverage Corp.")
print("Expiry: June 18th")
print("CRR European Call Price: $" + "{:3.2f}".format(crr_price))
print("B-S European Call Price: $" + "{:3.2f}".format(bs_price))

# Carvana 
crr_price = crr_trinomial_tree(265.32, 267.50, 0.00063, 15/252, n, 0.6065, 1)
bs_price  =     black_scholes(265.32, 267.50, 0.00063, 15/252, n, 0.6065, 1)
print("________________________________________________________________\n")
print("Carvana Co.")
print("Expiry: April 16th")
print("CRR European Call Price: $" + "{:3.2f}".format(crr_price))
print("B-S European Call Price: $" + "{:3.2f}".format(bs_price))

# American Airlines
crr_price = crr_trinomial_tree(23.86, 24.00, 0.006, 15/252, n, 0.5234, 1)
bs_price  =     black_scholes(23.86, 24.00, 0.006, 15/252, n, 0.5234, 1)
print("________________________________________________________________\n")
print("American Airlines Group Inc.")
print("Expiry: April 23rd (15 days)")
print("European Call Price: $" + "{:3.2f}".format(crr_price))
print("B-S European Call Price: $" + "{:3.2f}".format(bs_price))
print("________________________________________________________________\n")

# Spotify
crr_price = crr_trinomial_tree(273.10, 298.00, 0.006, 100/252, n, 0.5093, 1)
bs_price  =     black_scholes(273.10, 298.00, 0.006, 100/252, n, 0.5093, 1)
print("Spotify")
print("Expiry: April 23rd (15 days)")
print("European Call Price: $" + "{:3.2f}".format(crr_price))
print("B-S European Call Price: $" + "{:3.2f}".format(bs_price))
print("________________________________________________________________\n")


print("American Airlines:\n")
print("Steps  CRR    B-S")
for steps in range(25, 300, 25):
    crr_price = crr_trinomial_tree(23.86, 24.00, 0.006, 15/252, steps, 0.5234, 1)
    bs_price  =      black_scholes(23.86, 24.00, 0.006, 15/252, steps, 0.5234, 1)
    print("{:4.0f}".format(steps), "{:6.3f}".format(crr_price), "{:6.3f}".format(bs_price))
    


________________________________________________________________

Monster Beverage Corp.
Expiry: June 18th
CRR European Call Price: $5.77
B-S European Call Price: $5.78
________________________________________________________________

Carvana Co.
Expiry: April 16th
CRR European Call Price: $14.63
B-S European Call Price: $14.65
________________________________________________________________

American Airlines Group Inc.
Expiry: April 23rd (15 days)
European Call Price: $1.15
B-S European Call Price: $1.15
________________________________________________________________

Spotify
Expiry: April 23rd (15 days)
European Call Price: $25.46
B-S European Call Price: $25.50
________________________________________________________________

American Airlines:

Steps  CRR    B-S
  25  1.129  1.153
  50  1.142  1.153
  75  1.147  1.153
 100  1.148  1.153
 125  1.150  1.153
 150  1.150  1.153
 175  1.151  1.153
 200  1.151  1.153
 225  1.151  1.153
 250  1.152  1.153
 275  1.152  1.153


## Testing Markdown with latex 

This is an equation: 

$\hat{Y} = \hat{\beta}_{0} + \sum \limits _{j=1} ^{p} X_{j}\hat{\beta}_{j} $

$ F(0,t,T) = \mathbb{E}[r_t] + \frac{\sigma^2}{2}$